In [1]:
%%writefile mapper.py

import sys
import re
from math import floor

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        k, c = line.strip().split('\t', 1)
        c = int(floor(c))
    except ValueError as e:
        continue
    if not(current_key == k):
        if not(current_key == None):
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = k
    word_sum += c

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer.py


In [3]:
%%writefile mapper_2.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        w, k = unicode(line.strip()).split('\t', 1)
        k = int(floor(k))
    except ValueError as exception_2:
        continue
    print "%d\t%s" % (k, w)

Overwriting mapper_2.py


In [4]:
%%writefile reducer_2.py

import sys

for line in sys.stdin:
    try:
        c, k = line.strip().split('\t', 1)
        c = int(floor(c))
    except ValueError as exception_1:
        continue
    print "%s\t%d" % (key, count)

Overwriting reducer_2.py


In [5]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [6]:
%%bash
NUM_REDUCERS=8
OUT_DIR="wordcount_result_"$(date +"%s%6N")
#OUT_DIR="wordcount_result_nzueva

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

NEW_INPUT_DIR=${OUT_DIR}
OUT_DIR="wordrating_result_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${NEW_OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordRating" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options=-nr \
    -files mapper_2.py,reducer_2.py \
    -mapper "python mapper_2.py" \
    -reducer "python reducer_2.py" \
    -input ${NEW_INPUT_DIR} \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head -7 | tail -1

rm: `wordcount_result_1514402440076741': No such file or directory
17/12/27 19:20:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/27 19:20:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/27 19:20:44 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/27 19:20:45 INFO mapreduce.JobSubmitter: number of splits:2
17/12/27 19:20:45 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1514385445263_0008
17/12/27 19:20:45 INFO impl.YarnClientImpl: Submitted application application_1514385445263_0008
17/12/27 19:20:45 INFO mapreduce.Job: The url to track the job: http://a31ecc5c9e0d:8088/proxy/application_1514385445263_0008/
17/12/27 19:20:45 INFO mapreduce.Job: Running job: job_1514385445263_0008
17/12/27 19:20:50 INFO mapreduce.Job: Job job_1514385445263_0008 running in uber mode : false
17/12/27 19:20:50 INFO mapreduce.Job:  map 0% reduce 0%
17/12/27 19:21:06 INFO mapreduce.Job:  map 29% reduce 0%
17/12/27 19:2